# Interactive Jupyter-Speckle-Viewer

First we initiate the notebook requirements and some environment variables to hide the secrets. 🤫

In [1]:
%%capture 
# capture turns off the output for this cell which is the install log.
%pip install specklepy
%pip install ipywidgets

# plugin for loading environment variables from a .env file
%reload_ext dotenv 
%dotenv

HOST_SERVER=%env HOST_SERVER
TOKEN=%env XYZ_ACCESS_TOKEN

From there we run the boilerplate for a basic `specklepy` script

In [2]:
import ipywidgets as widgets
from ipywidgets import interact
from specklepy.api.client import SpeckleClient
from IPython.display import IFrame
import logging
from specklepy.api.models import Commit

client = SpeckleClient(host=HOST_SERVER)
client.authenticate_with_token(TOKEN)


Now we have an authenticated client, we can build the rudimentary `speckle-ui` using IPython widgets.

In [3]:
# The Streams selector is fed with the list of streams from the authenticated
# client.

select = [("Select a Stream", "")]
stream_list = client.stream.list()
stream_options = list(map(lambda x: (x.name, x.id), stream_list))

stream_widget = widgets.Dropdown(
    options=select + stream_options,
    description="Select a stream",
    style=dict(description_width="initial"),
    value=select[0][1],
    layout={"width": "max-content"},  # If the items' names are long\n"
)

# The Branch and Commit selectors will be populated following
# selection from the Streams selector.
branch_widget = widgets.Dropdown(
    description="Select a branch",
    style=dict(description_width="initial"),
    options=["Select a stream first"],
    disabled=True,
)

commit_widget = widgets.Dropdown(
    description="Select a commit",
    style=dict(description_width="initial"),
    options=[("Select a branch first", None)],
    disabled=True,
)

out = widgets.Output(layout={})

iframe = IFrame(
    src=f"https://",
    width="700px",
    height="500px",
    style={"border": "none"},
)


The following functions are part of the IPythonWidgets secret sauce, essentially adding an observer event to the selected values of Stream and then Branch and populating the options property of the already created widgets.

In [4]:
def on_stream_value_change(change):
    if change["type"] == "change" and change["name"] == "value":
        if change["new"] != "":
            branch_list = client.branch.list(stream_id=change["new"])
            branch_widget.options = ["Select a branch"] + list(
                map(lambda x: (x.name), branch_list)
            )
            branch_widget.disabled = False
        else:
            branch_widget.options = ["Select a stream first"]
            branch_widget.disabled = True

        branch_widget.value = branch_widget.options[0]


def on_branch_value_change(change):
    if change["type"] == "change" and change["name"] == "value":

        if (
            change["new"] == "Select a branch"
            or change["new"] == "Select a stream first"
        ):
            commit_widget.options = ["Select a branch first"]
            out.clear_output()
            commit_widget.disabled = True
        else:
            branch = client.branch.get(
                stream_id=stream_widget.value, name=change["new"]
            )

            if branch.commits.totalCount == 0:
                commit_widget.options = [("No commits yet on this branch", "")]
                out.clear_output()
                commit_widget.disabled = True
            else:
                commit_widget.options = [("Select a commit", "")] + list(
                    map(lambda c: (f"{c.message} ({c.id})", c.id), branch.commits.items)
                )
                commit_widget.disabled = False


# The Streams selector will update the Branch and Commit selectors.
# The on_displayed event is a nicety.
stream_widget.observe(on_stream_value_change)
branch_widget.observe(on_branch_value_change)


And its finally time to disply the interactive form. The embedded speckle
viewer will display if there is a valid commit.

In [5]:
# This decorator will make the widget interactive and will pass the selected
# value of each to the function.
@interact(
    stream_value=stream_widget, branch_value=branch_widget, commit_value=commit_widget
)
def get_commit(stream_value, branch_value, commit_value):
    if (
        commit_value == ""
        or commit_value is None
        or commit_value == "Select a commit"
        or commit_value == "Select a branch first"
        or commit_value == "No commits yet on this branch"
    ):
        out.clear_output()
        return

    out.clear_output(True)
    commit_object = client.commit.get(stream_id=stream_value, commit_id=commit_value)
    if commit_object.id:
        with out:
            iframe.src = f"https://{HOST_SERVER}/embed?stream={stream_value}&commit={commit_value}&filter=%7B%22colorBy%22%3A%7B%22type%22%3A%22category%22%2C%22property%22%3A%22Properties.Element.Stanhope%20Asset%20Reference%22%7D%7D"
            display(iframe)
    else:
        out.clear_output()
        with out:
            print("Something went wrong.")
    pass


display(out)


interactive(children=(Dropdown(description='Select a stream', layout=Layout(width='max-content'), options=(('S…

Output()